## Lecture 9: Input Pipeline
### 0. Overview
- Queues and Coordinators
- Data Readers Revisited
- TFRecord
- Variable Initializer
- Graph Collection
- Style Transfer

### 1. Queues and Coordinators
#### 1.1 Queues: for computing tensors asynchronously in a graph
advantages:
- Multiple threads prepare training examples and push them in the queue
- A training thread executes a training op that dequeues mini-batches from the queue

challenges:
- All threads must be able to stop together
- Exceptions must be caught and reported
- Queues must be properly closed when stopping.

**TensorFlow queues can’t run without proper threading,
but threading isn’t exactly pleasant in Python **  

#### 1.2 two classes to help with the threading: 
- `tf.Coordinator`: helps multiple threads stop together and report exceptions to a program that waits for them to stop
- `tf.train.QueueRunner`: create a number of threads cooperating to enqueue tensors in the same queue

#### 1.3 queue classes
![queueclass](figures/09_01.png)

- common practice: enqueue many examples in when reading data, but dequeue them one by one
- to get multiple elements at once for your batch training: use `tf.train.batch` or `tf.train.shuffle_batch` if you want to your batch to be shuffled.

#### 1.4 Queue is always used with string_input_producer

#### 1.5 example
09_queue_example.py

In [ ]:
N_SAMPLES = 1000
NUM_THREADS = 4

# Generating some simple data
# create 1000 random samples, each is a 1D array from the normal distribution (10, 1)
data = 10 * np.random.randn(N_SAMPLES, 4) + 1

# create 1000 random labels of 0 and 1
target = np.random.randint(0, 2, size=N_SAMPLES)

queue = tf.FIFOQueue(capacity=50, dtypes=[tf.float32, tf.int32], shapes=[[4], []])

# create ops that do something with data_sample and label_sample
enqueue_op = queue.enqueue_many([data, target])
data_sample, label_sample = queue.dequeue()

# create NUM_THREADS to do enqueue
qr = tf.train.QueueRunner(queue, [enqueue_op] * NUM_THREADS)


with tf.Session() as sess:
    # Create a coordinator, launch the queue runner threads.
    coord = tf.train.Coordinator()
    enqueue_threads = qr.create_threads(sess, coord=coord, start=True)
    for step in xrange(100): # do to 100 iterations
        if coord.should_stop():
            break
    data_batch, label_batch = sess.run([data_sample, label_sample])
    coord.request_stop()
    coord.join(enqueue_threads)
        


more examples in CS 110

### 2. Data Readers
previously mentioned in [Lecture 5](05_Managing experiments and process data.ipynb)

#### 2.1 `tf.train.string_input_producer`: create a queue to hold the names of all the files to be read in

In [ ]:
import tensorflow as tf
filename_queue = tf.train.string_input_producer(["heart.csv"])
reader = tf.TextLineReader(skip_header_lines=1)
    # it means you choose to skip the first line for every file in the queue

Think of readers as ops that return a different value every time you call it -- similar to Python generators
- key: a key to identify the file and record (useful for debugging if you have some weird records)
- value: a scalar string value

In [ ]:
key, value = reader.read(filename_queue)

For each example, the call to read() above might return:

    key = data/heart.csv:2
    value = 144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
    #  value “144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1” is the second line
    # (excluding the header line) in the file heart.csv

`tf.train.string_input_producer` creates a FIFOQueue under the hood  
-> need `tf.Coordinator` and `tf.QueueRunner` to run the queue

#### 2.2  run the queue

In [ ]:
filename_queue = tf.train.string_input_producer(filenames)
reader = tf.TextLineReader(skip_header_lines=1) # skip the first line in the file
key, value = reader.read(filename_queue)
with tf.Session() as sess:
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    print(sess.run(key)) # data/heart.csv:2
    print(sess.run(value)) # 144,0.01,4.41,28.61,Absent,55,28.87,2.06,63,1
    coord.request_stop()
    coord.join(threads)

#### 2.3 TensorFlow CSV decoder: convert string into vector

In [ ]:
content = tf.decode_csv(value, record_defaults=record_defaults)

- will parse value into the tensor record defaults which we have to create ourselves
- the record defaults serve two purposes
    - tells the decoder what **types** of data to expect in each column
    - in case of missing data in file, will fill in that space with the **default value of the data type** that we specify
    

In [ ]:
# an example of record defaults
record_defaults = [[1.0] for _ in range(N_FEATURES)] # define all features to be floats
record_defaults[4] = [''] # make the fifth feature string
record_defaults.append([1]) # make last column (label) integer
content = tf.decode_csv(value, record_defaults=record_defaults)

#### 2.4  data pre-processing
single sample:

In [ ]:
# convert the 5th column (present/absent) to the binary value 0 and 1
condition = tf.equal(content[4], tf.constant('Present'))
content[4] = tf.select(condition, tf.constant(1.0), tf.constant(0.0))
# pack all 9 features into a tensor
features = tf.pack(content[:N_FEATURES])
# assign the last column to label
label = content[-1]

batch ‘em up:

In [ ]:
# minimum number elements in the queue after a dequeue, used to ensure
# that the samples are sufficiently mixed
# I think 10 times the BATCH_SIZE is sufficient
min_after_dequeue = 10 * BATCH_SIZE
# the maximum number of elements in the queue
capacity = 20 * BATCH_SIZE
# shuffle the data to generate BATCH_SIZE sample pairs
data_batch, label_batch = tf.train.shuffle_batch([features, label], batch_size=BATCH_SIZE,
                                                 capacity=capacity, min_after_dequeue=min_after_dequeue)

full code see [Github Repo](https://github.com/chiphuyen/tf-stanford-tutorials/blob/master/examples/05_csv_reader.py)

### 3. TFRecord: TensorFlow's binary data format
A TFRecord is a serialized `tf.train.Example` Protobuf object. 

#### 3.0 why binary data?
- make better use of disk cache
- faster to move around
- can store data of different types (so you can put both images and labels in one place)

#### 3.1 read in the image and convert it to byte string

In [ ]:
def get_image_binary(filename):
    image = Image.open(filename)
    image = np.asarray(image, np.uint8)
    shape = np.array(image.shape, np.int32)
    return shape.tobytes(), image.tobytes() # convert image to raw data bytes in the array

#### 3.2 write the byte strings into a TFRecord file
Use `tf.python_io.TFRecordWriter` and `tf.train.Features`  
**Note:** Need the shape information so you can reconstruct the image from the binary format later.

In [ ]:
def write_to_tfrecord(label, shape, binary_image, tfrecord_file):
    """ This example is to write a sample to TFRecord file. If you want to write
    more samples, just use a loop.
    """
    writer = tf.python_io.TFRecordWriter(tfrecord_file)
    # write label, shape, and image content to the TFRecord file
    example = tf.train.Example(features=tf.train.Features(feature={
        'label': tf.train.Feature(bytes_list=tf.train.BytesList(value=[label])),
        'shape': tf.train.Feature(bytes_list=tf.train.BytesList(value=[shape])),
        'image':tf.train.Feature(bytes_list=tf.train.BytesList(value=[binary_image]))
    }))
    writer.write(example.SerializeToString())
    writer.close()

#### 3.3  read a TFRecord file
Use `TFRecordReader` and `tf.decode_raw`

In [ ]:
def read_from_tfrecord(filenames):
    tfrecord_file_queue = tf.train.string_input_producer(filenames, name='queue')
    reader = tf.TFRecordReader()
    _, tfrecord_serialized = reader.read(tfrecord_file_queue)
    
    # label and image are stored as bytes but could be stored as
    # int64 or float64 values in a serialized tf.Example protobuf.
    tfrecord_features = tf.parse_single_example(tfrecord_serialized,
                                                features={
                                                    'label': tf.FixedLenFeature([], tf.string),
                                                    'shape': tf.FixedLenFeature([], tf.string),
                                                    'image': tf.FixedLenFeature([], tf.string),
                                                }, name='features')
    # image was saved as uint8, so we have to decode as uint8.
    image = tf.decode_raw(tfrecord_features['image'], tf.uint8)
    shape = tf.decode_raw(tfrecord_features['shape'], tf.int32)
    
    # the image tensor is flattened out, so we have to reconstruct the shape
    image = tf.reshape(image, shape)
    label = tf.cast(tfrecord_features['label'], tf.string)
    return label, shape, image

**Note:**  `label`, `shape`, and `image` returned are tensor objects. To get their values, you’ll have to eval them in `tf.Session()`

### 4. Style Transfer
[paper](https://arxiv.org/pdf/1701.04928v1.pdf)  

#### 4.1 goal explained
Find a new image:  
- whose content is closest to the content image and
- whose style is closest to the style image

#### 4.2 What is the content/style of an image?
Feature visualization have shown that:
- lower layers extract features related to content
- higher layers extract features related to style

#### 4.3 all about the loss functions:
- **Content loss**: $L_{content}(\vec{p}, \vec{x},l)=\frac{1}{2} \sum_{i,j}(F_{ij}^l-P_{ij}^l)^2$
    - To measure the content loss between the content (**the feature map in the content layer**) of the generated image and the content of the content image
    - Paper: `'conv4_4'`
- **Style loss**: $L_{style}(\vec{a}, \vec{x})=\sum_{l=0}^L w_lE_l,\qquad E_l=\frac{1}{4N_l^2 M_l^2}\sum_{i,j}(G_{ij}^l-A_{ij}^l)^2$
    - To measure the style loss between the style (**the feature maps in the style layers ->  the gram matrices of feature maps in the style layers**) of the generated image and the style of the style image
    - Paper: `[‘conv1_1’, ‘conv2_1’, ‘conv3_1’, ‘conv4_1’ and ‘conv5_1’]`
    - Give more weight to deeper layers: E.g. 1.0 for `‘conv1_1’`, 2.0 for `‘conv2_1’`, ...

#### 4.4 optimizer
Optimizes the **initial image** (not the weights!) to minimize the combination of the two losses  
$$L_{total}(\vec{p}, \vec{a}, \vec{x})= \alpha L_{content}(\vec{p}, \vec{x})+ \beta L_{style}(\vec{a}, \vec{x})$$

#### 4.5 Tricky implementation details
- Train input instead of weights
- Multiple tensors share the same variable to avoid assembling identical subgraphs
    - content image
    - style image
    - initial image
- Use pre-trained weights (from VGG-19)
    - Weights and biases already loaded for you
    - They are numpy, so need to be converted to tensors
    - Must not be trainable!!